In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/knowledge-graph/kelm/train_X
# !wget https://f000.backblazeb2.com/file/malay-dataset/knowledge-graph/kelm/train_Y

In [2]:
from tqdm import tqdm
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [4]:
with open('train_X') as fopen:
    train_X = fopen.read().split('\n')
    
with open('train_Y') as fopen:
    train_Y = fopen.read().split('\n')

In [5]:
top = 2000000

with tf.io.gfile.GFile('knowledge-graph.tsv', "w") as outfile:
    for i in tqdm(range(top)):
        if len(train_X) and len(train_Y):
            l = cleaning(train_X[i])
            r = cleaning(train_Y[i])
            outfile.write("%s\t%s\n" % (l, r))

100%|██████████| 2000000/2000000 [00:59<00:00, 33669.22it/s]


In [6]:
def knowledge_graph_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'knowledge-graph.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def knowledge_graph_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['grafik pengetahuan: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [7]:
t5.data.TaskRegistry.remove('knowledge_graph_dataset')
t5.data.TaskRegistry.add(
    'knowledge_graph_dataset',
    dataset_fn = knowledge_graph_dataset,
    splits = ['train'],
    text_preprocessor = [knowledge_graph_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


In [8]:
nq_task = t5.data.TaskRegistry.get("knowledge_graph_dataset")
ds = nq_task.get_dataset(split='knowledge-graph.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [9]:
next(r)

{'inputs_plaintext': b'grafik pengetahuan: Keuskupan Agung Katolik Rom Maracaibo terletak di barat daya Keuskupan Katolik Rom Machiques.',
 'inputs': array([12333,  5836,    31,   386,   450,   848,  1744,  3526,  7751,
         5503,  9519,   833,    91,  1079,   282,    24,  1940,  2682,
          386,   450,   848,  1744,  7751,  5503, 19940, 12633,    16,
            3,     1]),
 'targets_plaintext': b'Roman Catholic Archdiocese of Maracaibo shares border with Roman Catholic Diocese of Machiques, Roman Catholic Diocese of Machiques direction relative to location southwest.',
 'targets': array([ 5000,  7253, 11743, 10469,   841,   478,    18,  9519,   833,
           91,  1079,  3128,  3596,    40,  5000,  7253,   208,  4172,
         5445,    18, 19940, 12633,    16,    14,  5000,  7253,   208,
         4172,  5445,    18, 19940, 12633,    16,  4222, 11055,    19,
         4538, 14542,     3,     1])}